In [1]:
# 0 define backend
import sys, os, time

# %env DDE_BACKEND=tensorflow.compat.v1
# %env XLA_FLAGS=--xla_gpu_cuda_data_dir=/usr/local/home/cyan3/miniforge/envs/tf

os.environ['DDE_BACKEND'] = "pytorch" # v2
os.environ['XLA_FLAGS'] = "--xla_gpu_cuda_data_dir=/usr/local/home/cyan3/miniforge/envs/tf"

# https://stackoverflow.com/questions/68614547/tensorflow-libdevice-not-found-why-is-it-not-found-in-the-searched-path
# this directory has /nvvm/libdevice/libdevice.10.bc

print(os.environ['DDE_BACKEND'])

pytorch


In [4]:
# import tensorflow as tf
# gpu_devices = tf.config.experimental.list_physical_devices('GPU')
# for device in gpu_devices:
#     print(device)

import torch
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))

import deepxde as dde
import numpy as np
from numpy import linalg as LA
import math

import matplotlib.pyplot as plt
import pylab

from os.path import dirname, join as pjoin

from scipy import stats
import scipy.io
from scipy.stats import truncnorm, norm
from scipy.optimize import linprog
from scipy import sparse
from scipy.stats import multivariate_normal

if dde.backend.backend_name == "pytorch":
    exp = dde.backend.torch.exp
else:
    from deepxde.backend import tf

    exp = tf.exp
    
import cvxpy as cp
import numpy as np
import tensorflow as tf
from scipy.linalg import solve_discrete_are
from scipy.linalg import sqrtm
from cvxpylayers.tensorflow.cvxpylayer import CvxpyLayer

print(time.time())

True
1
NVIDIA RTX A2000 Laptop GPU


2022-09-10 16:14:18.535078: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-10 16:14:18.618345: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-09-10 16:14:18.637725: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-09-10 16:14:18.967326: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

1662851659.4038758


In [5]:
N = nSample = 100

# must be floats
state_min = 0.0
state_max = 6.0

j1, j2, j3 =1,1,2 # axis-symmetric case
q_statepenalty_gain = 0 # 0.5

T_0=0. #initial time
T_t=200. #Terminal time

print(time.time())

1662851666.1446936


In [6]:
x_T = np.transpose(np.linspace(state_min, state_max, N))
y_T = np.transpose(np.linspace(state_min, state_max, N))
z_T = np.transpose(np.linspace(state_min, state_max, N))
x_T=x_T.reshape(len(x_T),1)
y_T=y_T.reshape(len(y_T),1)
z_T=z_T.reshape(len(z_T),1)

print(time.time())

1662851669.0200875


In [5]:
def pdf1d(x, mu, sigma):
    a, b = (state_min - mu) / sigma, (state_max - mu) / sigma
    rho_x=truncnorm.pdf(x, a, b, loc = mu, scale = sigma)

    # do NOT use gaussian norm, because it is only area=1
    # from -inf, inf, will not be for finite state/grid
    # rho_x = norm.pdf(x, mu, sigma)
    return rho_x

def boundary(_, on_initial):
    return on_initial

print(time.time())

1662833150.55448


In [6]:
x_grid = np.transpose(np.linspace(state_min, state_max, nSample))
y_grid = np.transpose(np.linspace(state_min, state_max, nSample))
[X,Y] = np.meshgrid(x_grid,x_grid)
C = (X - Y)**2

# cvector = C.flatten('F')
cvector = C.reshape(nSample**2,1)

A = np.concatenate(
    (
        np.kron(
            np.ones((1,nSample)),
            sparse.eye(nSample).toarray()
        ),
        np.kron(
            sparse.eye(nSample).toarray(),
            np.ones((1,nSample))
        )
    ), axis=0)
# 2*nSample

print(cvector.shape)
print(A.shape)

(10000, 1)
(200, 10000)


In [7]:
# 1d linprog example

rho_0_1d=pdf1d(x_T, 5.0, 1.0).reshape(len(x_T),1)
rho_T_1d=pdf1d(x_T, 4.0, 1.0).reshape(len(x_T),1)

rho_0_1d = np.where(rho_0_1d < 0, 0, rho_0_1d)
rho_0_1d = rho_0_1d / np.sum(np.abs(rho_0_1d))

# rho_0_1d_trapz = np.trapz(rho_0_1d, axis=0)[0]
# rho_0_1d = rho_0_1d / rho_0_1d_trapz
# rho_0_1d_trapz = np.trapz(rho_0_1d, axis=0)
# print("rho_0_1d_trapz=",rho_0_1d_trapz)

rho_T_1d = np.where(rho_T_1d < 0, 0, rho_T_1d)
rho_T_1d = rho_T_1d / np.sum(np.abs(rho_T_1d))

# rho_T_1d_trapz = np.trapz(rho_T_1d, axis=0)[0]
# rho_T_1d = rho_T_1d / rho_T_1d_trapz
# rho_T_1d_trapz = np.trapz(rho_T_1d, axis=0)
# print("rho_T_1d_trapz=",rho_T_1d_trapz)

rho_0 = rho_0_1d
rho_T = rho_T_1d

print(time.time())

1662833152.70004


In [8]:
res = linprog(
    cvector,
    A_eq=A,
    b_eq=np.concatenate((rho_0, rho_T), axis=0),
    bounds=[(0, np.inf)],
    options={"disp": True}
)
print(res.fun)

Running HiGHS 1.2.2 [date: 2022-08-26, git hash: n/a]
Copyright (c) 2022 ERGO-Code under MIT licence terms
Presolving model
200 rows, 10000 cols, 20000 nonzeros
199 rows, 10000 cols, 19900 nonzeros
Presolve : Reductions: rows 199(-1); columns 10000(-0); elements 19900(-100)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 199(1.99666) 0s
       2699     6.3187344716e-01 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 2699
Objective value     :  6.3187344716e-01
HiGHS run time      :          0.12
0.6318734471629914


In [13]:
# Define and solve the CVXPY problem.
x = cp.Variable(
    cvector.shape[0],
    nonneg=True
)
prob = cp.Problem(
    cp.Minimize(cvector.T @ x),
    [
        A @ x == np.concatenate((
            rho_0,
            rho_T
        ), axis=0).reshape(-1),
        # do NOT specify bounds constraints here
    ],
)
prob.solve(verbose=False)

# Print result.
print("\nThe optimal value is", prob.value)

# print("A dual solution is")
# print(prob.constraints[0].dual_value)


The optimal value is 0.6318734626810212


In [10]:
import cvxpy as cp
import tensorflow as tf
from cvxpylayers.tensorflow import CvxpyLayer

# Define and solve the CVXPY problem.
x = cp.Variable(
    cvector.shape[0],
    nonneg=True
)
pred = cp.Parameter((A.shape[0],))
problem = cp.Problem(
    cp.Minimize(cvector.T @ x),
    [
        A @ x == pred,
    ],
)
assert problem.is_dpp()
cvxpylayer = CvxpyLayer(
    problem,
    parameters=[pred],
    variables=[x])

print(time.time())

1662833159.6947536


In [17]:
rho_0_tf = tf.constant(rho_0, shape=(100,))

rho_I=pdf1d(x_T, 0.0, 0.1).reshape(len(x_T),1)
rho_I = np.where(rho_I < 0, 0, rho_I)
rho_I = rho_I / np.sum(np.abs(rho_I))
rho_I = rho_I.reshape((100,))

# y_pred = tf.Variable(tf.zeros((100,), tf.double))
y_pred = tf.Variable(rho_I, shape=(100,))
# this MUST be tf.Variable to get a gradient w.r.t

print(time.time())

1662833551.6717205


In [19]:
with tf.GradientTape() as tape:
  # solve the problem, setting the values of A, b to A_tf, b_tf
  param = tf.concat([rho_0_tf, y_pred], 0)
  print(type(param))
  x_sol, = cvxpylayer(param)
  wass_dist1 = tf.tensordot(cvector.T, x_sol, 1)
#   wass_dist2 = tf.matmul(cvector2, x_sol)

wass_dist3 = cvector.T @ x_sol.numpy()

print("wass_dist1=", wass_dist1)
# print("wass_dist2=", wass_dist2)
print("wass_dist3=", wass_dist3)

# compute the gradient of the summed solution with respect to A, b
grad_ypred = tape.gradient(wass_dist1, [y_pred])
print("grad_ypred", grad_ypred[0].numpy())

<class 'tensorflow.python.framework.ops.EagerTensor'>
wass_dist1= tf.Tensor([22.278], shape=(1,), dtype=float64)
wass_dist3= [22.278]
grad_ypred [ 19.06   18.513  17.885  17.213  16.519  15.825  15.138  10.144   0.138  -0.796  -1.122  -1.413  -1.696  -1.972  -2.24   -2.5    -2.754  -2.999  -3.237  -3.467  -3.69   -3.906  -4.115
  -4.316  -4.509  -4.696  -4.876  -5.049  -5.214  -5.372  -5.524  -5.668  -5.805  -5.935  -6.057  -6.172  -6.28   -6.38   -6.473  -6.559  -6.638  -6.71   -6.775  -6.833  -6.883  -6.926
  -6.962  -6.99   -7.18   -7.533  -7.88   -8.223  -8.562  -8.894  -9.221  -9.54   -9.853 -10.159 -10.461 -10.757 -11.046 -11.328 -11.603 -11.871 -12.131 -12.385 -12.631 -12.869 -13.101
 -13.325 -13.541 -13.751 -13.953 -14.148 -14.335 -14.515 -14.687 -14.853 -15.011 -15.161 -15.305 -15.44  -15.569 -15.69  -15.804 -15.911 -16.01  -16.102 -16.186 -16.263 -16.333 -16.395
 -16.451 -16.498 -16.539 -16.572 -16.597 -16.616 -16.627 -16.631]


In [20]:
# this is their example, where you take a gradient w.r.t a Parameter


# Generate data
tf.random.set_seed(1)
np.random.seed(1)

n = 2
m = 3

A = np.eye(n) + 1e-2 * np.random.randn(n, n)
B = 1e-2 / 3 * np.random.randn(n, m)
Q = np.eye(n)
R = np.eye(m)

# Compute LQR control policy
P_lqr = solve_discrete_are(A, B, Q, R)
P = R + B.T@P_lqr@B
P_sqrt_lqr = sqrtm(P)

# Construct CVXPY problem and layer
x_cvxpy = cp.Parameter((n, 1))
P_sqrt_cvxpy = cp.Parameter((m, m))
P_21_cvxpy = cp.Parameter((n, m))
q_cvxpy = cp.Parameter((m, 1))

u_cvxpy = cp.Variable((m, 1))
y_cvxpy = cp.Variable((n, 1))

objective = .5 * cp.sum_squares(P_sqrt_cvxpy @ u_cvxpy) + x_cvxpy.T @ y_cvxpy + q_cvxpy.T @ u_cvxpy
problem = cp.Problem(
    cp.Minimize(objective),
    [
        cp.norm(u_cvxpy) <= 1,
         y_cvxpy == P_21_cvxpy @ u_cvxpy])
assert problem.is_dpp()

policy = CvxpyLayer(
    problem,
    [x_cvxpy, P_sqrt_cvxpy, P_21_cvxpy, q_cvxpy],
    [u_cvxpy]
)

def train(iters):
    # Initialize with LQR control lyapunov function
    P_sqrt = tf.Variable(P_sqrt_lqr)
    P_21 = tf.Variable(A.T @ P_lqr @ B)
    q = tf.Variable(tf.zeros((m, 1), dtype=tf.float64))
    variables = [P_sqrt, P_21, q]
    A_tf, B_tf, Q_tf, R_tf = map(tf.constant, [A, B, Q, R])

    def g(x, u):
        return tf.squeeze(
            tf.transpose(x) @ Q_tf @ x + tf.transpose(u) @ R_tf @ u)

    def evaluate(x0, P_sqrt, P_21, q, T):
        x = x0
        cost = 0.
        for _ in range(T):
            print(type(P_sqrt), type(P_21), type(q), type(x))
            u, = policy(x, P_sqrt, P_21, q)
            cost += g(x, u) / T
            x = A_tf @ x + B_tf @ u + .2 * tf.random.normal((n, 1), dtype=tf.float64)
        return cost

    def eval_loss(N=8, T=25):
        return sum(
        [
            evaluate(
                tf.zeros((n, 1), dtype=tf.float64),
                P_sqrt,
                P_21,
                q,
                T=T
            ) for _ in range(N)
        ]) / N

    results = []
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.02, momentum=0.9)
    for i in range(iters):
        tf.random.set_seed(1)
        np.random.seed(1)
        with tf.GradientTape() as tape:
            loss = eval_loss()
        gradients = tape.gradient(loss, variables)
        optimizer.apply_gradients(zip(gradients, variables))
        results.append(loss.numpy())
        print("(iter %d) loss: %g " % (i, results[-1]))
    return results

results = train(iters=100)

import matplotlib.pyplot as plt

plt.figure()
plt.plot(results)
plt.xlabel('iteration')
plt.ylabel('average cost')
plt.show()

<class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.ops.resource_variable_ops.Resou

<class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.ops.resource_variable_ops.Resou

<class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.ops.resource_variable_ops.Resou

<class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.ops.resource_variable_ops.Resou

(iter 0) loss: 1.18636 
<class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.ops.res

...

<class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.ops.resource_variable_ops.Resou

<class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.ops.resource_variable_ops.Resou

<class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.ops.resource_variable_ops.Resou

(iter 14) loss: 0.850686 
<class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> <class 'tensorflow.python.ops.r

KeyboardInterrupt: 